In [1]:
import pickle
import json
import pymorphy2
import pandas as pd
from ner.network import NER
from ner.corpus import Corpus
from nltk.corpus import stopwords

In [2]:
with open("texts.pkl", "rb") as file:
    data = pickle.load(file)

In [3]:
data = data.reset_index(drop=True)

In [4]:
data.head()

,source,author,text,date
0,Lenta.ru,Unknown,Вопреки апокалиптическим прогнозам американски...,2000-01-01
1,Интерфакс,Unknown,Мадрид не выдаст Венесуэле укрывшегося в посо...,2000-01-01
2,Интерфакс,Unknown,Пожар на складе резины произошел в промзоне Н...,2000-01-01
3,Интерфакс,Unknown,Полиция Нью-Йорка обвинила трех подростков в ...,2000-01-01
4,Интерфакс,Unknown,Трамп направил кандидатуру постпреда США при ...,2000-01-01


In [5]:
morph = pymorphy2.MorphAnalyzer()

def tokenize(text):
    return ([word.strip('.,:;?!()""''') for word in text.split(" ")])

def lemmatize(text):
    return ([morph.parse(word)[0].normal_form for word in tokenize(text)])

In [6]:
with open('model/params.json') as f:
    network_params = json.load(f)

corpus = Corpus(dicts_filepath='model/dict.txt')

network = NER(corpus, verbouse=False, pretrained_model_filepath='model/ner_model', **network_params)

/Users/olgapichuzhkina/anaconda3/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:95: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:Restoring parameters from model/ner_model


In [8]:
text = lemmatize(data["text"][10000])
tags = network.predict_for_token_batch([text])[0]
print(', '.join([f"{word}: {tag}" for word, tag in zip(text, tags)]))

в: O, пятница: O, около: O, 23:10: O, в: O, дом: O, 20: O, на: O, улица: O, генерал: O, кузнецов: B-PER, в: O, москва: B-LOC, быть: O, убитый: O, генеральный: O, директор: O, ооо: B-ORG, андреас: I-ORG, игорь: B-PER, волобуев: I-PER, как: O, сообщать: O, риа: O, новость: O, неизвестный: O, преступник: O, трижды: O, выстрелить: O, в: O, 42-летний: O, бизнесмен: O, из: O, автомат: O, калашников: B-PER, с: O, глушитель: O, затем: O, он: O, произвести: O, четыре: O, контрольный: O, выстрел: O, в: O, голова: O, тело: O, убитый: O, обнаружить: O, в: O, кабина: O, лифт: O, он: O, жена: O, вестись: O, расследование: O, правоохранительный: O, орган: O, устанавливать: O, круг: O, связь: O, убитый: O, ооо: B-ORG, андреас: I-ORG, который: I-ORG, возглавлять: O, волобуев: O, специализироваться: O, на: O, продажа: O, недвижимость: O
